In [83]:
!git clone https://github.com/jacoponudo/Size_effects.git

fatal: destination path 'Size_effects' already exists and is not an empty directory.


In [84]:

root='/kaggle/working/'
import sys
module_path = root+'Size_effects/EXP'
sys.path.append(module_path)
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import random
from scipy.stats import chi2
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import pandas as pd
from EXP_package.functions import *
import tqdm 
!pip install fastparquet
!pip install gdown

^C
ERROR: Operation cancelled by user



KeyboardInterrupt



# Reddit

In [ ]:
# Scarica il dataset di Reddit
import gdown

url='https://drive.google.com/uc?id=1QepHehlhqP-jtOcshFzajqxj1DpJtoj7'
output='reddit.parquet'

gdown.download(url,output,quiet=False)
reddit = pd.read_parquet('/kaggle/working/reddit.parquet')

social=reddit

# Gab

In [ ]:
# Scarica il dataset di Gab
import gdown

url='https://drive.google.com/uc?id=1CpsRAaBVv4hIoq713KQmWYyZOlMG2BpH'
output='gab.parquet'

gdown.download(url,output,quiet=False)
gab = pd.read_parquet('/kaggle/working/gab.parquet')

social=gab
social['user_id']=social['user']

## Facebook

In [ ]:
# Scarica i tre dataset che appartengono a Facebook
import gdown
url='https://drive.google.com/uc?id=1Y2lGWkcgo_IWHdWFh_Qcn0K74D_xQhvB'
output='facebook_news.csv'
gdown.download(url,output,quiet=False)

url='https://drive.google.com/uc?id=1KGH2s8sPnvXZc7_wVgTCoGTXPr3B7SCe'
output='facebook_brexit.csv'
gdown.download(url,output,quiet=False)

url='https://drive.google.com/uc?id=17q4X4JHYJBz1jF58XvYfbKqZLy-jtmzM'
output='facebook_vaccines.csv'
gdown.download(url,output,quiet=False)

# Leggi i tre dataset con pandas
facebook_news = pd.read_csv('/kaggle/working/facebook_news.csv', dtype={'from_id': str})
facebook_vaccines_all_comments_scored = pd.read_csv('/kaggle/working/facebook_vaccines.csv', dtype={'from_id': str})
facebook_brexit_all_comments_scored = pd.read_csv('/kaggle/working/facebook_brexit.csv', dtype={'from_id': str})

# Etichetta e raggruppa i tre dataset
facebook_vaccines = facebook_vaccines_all_comments_scored.copy()
facebook_vaccines['topic'] = 'Vaccines'

facebook_brexit= facebook_brexit_all_comments_scored.copy()
facebook_brexit['topic'] = 'Brexit'

facebook_news = facebook_news.copy()
facebook_news['topic'] = 'News'

facebook = pd.concat([facebook_vaccines, facebook_brexit, facebook_news])
facebook.reset_index(drop=True, inplace=True)
facebook['user_id']=facebook['from_id']

social=facebook

# Gini index of Activity

Calcoliamo per ogni thread un coefficiente di Gini che misuri la concentrazione dell'attività degli utenti.

In [ ]:
# Conta i commenti per ogni post_id
N_df = social.groupby('post_id')['user_id'].nunique().reset_index()
N_df.columns = ['post_id', 'count']
           #N_df=N_df[N_df['count']>50]

# Filtra i post con meno di 
so=social[social['post_id'].isin(N_df['post_id'])]

# Calcola il coefficiente di gini sulla concentrazione dell'attività degli utenti 
gini_df = calculate_gini_for_posts(so)

# Unisci i dati sul post_id
merged_df = pd.merge(gini_df, N_df, on='post_id')

# Plot
plt.figure(figsize=(12, 6))
plt.scatter(merged_df['count'], merged_df['gini_index'], alpha=0.05)
plt.title('Indice di Gini per Post_ID')
plt.xlabel('Utenti per Post_ID')
plt.ylabel('Indice di Gini')
plt.xscale('log')  # Log scale for better visualization if the count ranges widely
plt.yscale('linear')  # Adjust scale as needed
plt.grid(True)
plt.show()


Boxplot del coefficiente di gini rispetto al numero di utenti in un thread

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Supponiamo che merged_df sia già definito e contenga 'count' e 'gini_index'
x = merged_df['count']
y = merged_df['gini_index']

# Creare tre classi omogenee basate sui quantili dei dati
quantiles = np.percentile(x, np.linspace(0, 100, 11))  # 10 classi
labels = [f'Classe {i+1}' for i in range(10)]  # Etichette per le 10 classi
merged_df['classe'] = pd.cut(x, bins=quantiles, labels=labels, include_lowest=True)

# Creare boxplot per le tre classi
plt.figure(figsize=(12, 6))
merged_df.boxplot(column='gini_index', by='classe', grid=False)
plt.title('Boxplot dell\'Indice di Gini per Classe')
plt.suptitle('')  # Rimuovere il titolo generato automaticamente
plt.xlabel('Classe di Conteggio dei Post')
plt.ylabel('Indice di Gini')
plt.show()



# Outreach della pagina 

In [134]:
gab

,comment_id,text,created_at,is_reply,parent_id,user,created_utc,is_quote,attachment,like_count,...,reply_count,score,conversation_parent_id,quote_conversation_parent_id,toxicity_score,post_id,social,topic,is_toxic,user_id
0,3305508,@JimDurfee Must b a coordinated #BLM melee fu...,2016-12-27 03:19:15+00:00,True,3304944.0,121303,1.482809e+09,False,"{'type': None, 'value': None}",1.0,...,0.0,1.0,NaN,NaN,0.111403,3304944,Gab,Feed,False,121303
1,3305514,@JaredWyand it is brutal to watch. So disappoi...,2016-12-27 03:19:29+00:00,True,3305201.0,63689,1.482809e+09,False,"{'type': None, 'value': None}",1.0,...,0.0,1.0,NaN,NaN,0.555970,3305201,Gab,Feed,False,63689
2,3305518,@CuriousOfTrump I am so sick of Muslim this an...,2016-12-27 03:19:36+00:00,True,3305198.0,102726,1.482809e+09,False,"{'type': None, 'value': None}",4.0,...,0.0,5.0,NaN,NaN,0.602753,3305198,Gab,Feed,True,102726
3,3305522,@memedna You can read us here www.whaleoil.co....,2016-12-27 03:19:38+00:00,True,3299916.0,6699,1.482809e+09,False,"{'type': None, 'value': None}",1.0,...,0.0,1.0,NaN,NaN,0.010681,3297093,Gab,Feed,False,6699
4,3305523,@JaredWyand See. This is something I can fully...,2016-12-27 03:19:40+00:00,True,3305201.0,111529,1.482809e+09,False,"{'type': None, 'value': None}",0.0,...,0.0,0.0,NaN,NaN,0.199166,3305201,Gab,Feed,False,111529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14641428,26560432,"Heâs African. Not to sound racist, but many ...",2018-05-29 09:15:40+00:00,True,26559733.0,175625,1.527585e+09,False,"{'type': None, 'value': None}",1.0,...,0.0,1.0,26559733.0,NaN,0.752083,26559733,Gab,Feed,True,175625
14641429,26560435,Been on that post fr a loooooonnnng time,2018-05-29 09:15:43+00:00,True,26560019.0,384953,1.527585e+09,False,"{'type': None, 'value': None}",0.0,...,0.0,0.0,26560019.0,NaN,0.142560,26560019,Gab,Feed,False,384953
14641430,26560436,"Oh sorry Lea, I was miles away watching a on-l...",2018-05-29 09:15:47+00:00,True,26560210.0,217163,1.527585e+09,False,"{'type': None, 'value': None}",1.0,...,0.0,1.0,26560210.0,NaN,0.112437,26559981,Gab,Feed,False,217163
14641431,26560449,Man that ain't right,2018-05-29 09:16:11+00:00,False,26559994.0,227650,1.527585e+09,True,"{'type': 'media', 'value': [{'height': 325, 'i...",0.0,...,0.0,0.0,26559994.0,NaN,0.053048,26559994,Gab,Feed,False,227650


In [131]:
# Aggiungiamo la colonna autore, ovvero id della pagina che ha prodotto il post
social['created_at'] = pd.to_datetime(social['created_at'])
primi_mess = social.loc[social.groupby('post_id')['created_at'].idxmin()]
authors = primi_mess[['post_id', 'user_id']].rename(columns={'user_id': 'autore'})
social = social.merge(authors, on='post_id', suffixes=('', 'autore'))

Parto da una lista di pagine, per ogniuna di queste calcolo per ogni post il numero di utenti coinvolti in generale dalla pagina in quel periodo

In [142]:
# Campiona 100 pagine
pages = social['autore'].sample(100)
outreach_results = []

for page in pages:
    # Seleziono tutti i post della pagina
    df = so[so['post_id'].isin(author_with_post_count[author_with_post_count['user_id'] == page]['post_id'].unique())]
    df['created_at'] = pd.to_datetime(df['created_at'])

    # Osservo la loro data di creazione
    post_dates = df.groupby('post_id')['created_at'].min().reset_index()
    post_dates.rename(columns={'created_at': 'post_date'}, inplace=True)

    # Conto quanti utenti hanno commentato almeno una volta un post della pagina in quel periodo
    post_dates['users_one_month_window'] = post_dates['post_date'].apply(lambda x: count_unique_users_within_period(x, 100))
    
    # Aggiungo i risultati alla lista
    outreach_results.append(post_dates[['post_id', 'users_one_month_window']])

outreach_df = pd.concat(outreach_results, ignore_index=True)


/tmp/ipykernel_36/2106926144.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_at'] = pd.to_datetime(df['created_at'])
/tmp/ipykernel_36/2106926144.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_at'] = pd.to_datetime(df['created_at'])
/tmp/ipykernel_36/2106926144.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

KeyboardInterrupt: 

In [145]:
outreach_df = pd.concat(outreach_results, ignore_index=True)
outreach_df

,post_id,users_one_month_window
0,32393181,38
1,32476278,41
2,32636412,44
3,32730907,47
4,32785051,56
...,...,...
236,37710534,67
237,37717447,67
238,37969760,67
239,38097770,67


## **Funcions** 

In [141]:
import pandas as pd
import numpy as np
from tqdm import tqdm
def gini_coefficient(values):
    if len(values) == 0:
        return np.nan  # O un valore che hai scelto per i casi senza dati
    sorted_values = np.sort(values)
    cumulative_values = np.cumsum(sorted_values)
    
    # Aggiungi un controllo per evitare problemi con array troppo piccoli
    if len(cumulative_values) == 1:
        return 0  # O un valore che hai scelto per i casi con un solo elemento
    
    lorenz_curve = cumulative_values / cumulative_values[-1]
    gini_index = 1 - 2 * np.trapz(lorenz_curve, dx=1/len(lorenz_curve))
    return gini_index


def calculate_gini_for_posts(df):
    """Calcola l'indice di concentrazione di Gini per ogni post_id basato sul numero di righe per utente."""
    gini_results = []
    
    # Raggruppa i dati per post_id
    grouped = df.groupby('post_id')
    
    for post_id, group in tqdm(grouped):
        # Conta il numero di righe per ogni utente
        user_counts = group['user_id'].value_counts()
        # Calcola l'indice di Gini per questi conteggi
        gini_index = gini_coefficient(user_counts.values)
        gini_results.append({'post_id': post_id, 'gini_index': gini_index})
    
    return pd.DataFrame(gini_results)

def count_unique_users_within_period(post_date, period):
    start_time = pd.to_datetime(str(post_date)) - pd.Timedelta(days=10)
    end_time = pd.to_datetime(str(post_date))  + pd.Timedelta(days=10)
    relevant_comments = df[(df['created_at'] >= start_time) & (df['created_at'] <= end_time)]
    unique_users = relevant_comments['user_id'].nunique()
    return unique_users